# Approach: Keep adding edges until no more can be added

## 1. Import

In [14]:
import networkx as nx
import pandas as pd
from sklearn.metrics import mean_squared_error

## 2. Load data

In [2]:
def loadTrainDataAsUndirectedGraph():
    filename = "train.txt"
    rows = [line.rstrip("\n") for line in open(filename)]
    g = nx.Graph()
    for row in rows:
        authorIds = row.split()
        for i, author in enumerate(authorIds):
            for coauthor in authorIds[i+1:]:
                if g.has_edge(author, coauthor):
                    g[author][coauthor]['frequency'] += 1
                else:
                    g.add_edge(author, coauthor, frequency=1)
    return g

In [3]:
trainGraph = loadTrainDataAsUndirectedGraph()

## 3. Train

In [4]:
class EdgeAddClassifier:
    def __init__(self):
        self.model = None
    
    def train(self, graph):
        self.__trainAttempt1(graph)
    
    def predict(self, edgeList):
        return self.__predictAttempt1(edgeList)
    
    #####################################################################
    # Attempt 1: Kaggle Score = 0.79480 (MSE = 0.2680352579128701)
    #####################################################################
    
    def __trainAttempt1(self, graph):
        self.__buildProbabilityGraph(graph)
        print('iteration 0: number of edges = ', len(self.model.edges))
        self.__addEdgesBasedOnProximity(4)
        print('iteration 1: number of edges = ', len(self.model.edges))    
    
    def __predictAttempt1(self, edgeList):
        return edgeList.apply(
            lambda i: self.__classifyInstance1(str(i.Source), str(i.Sink)),
            axis=1)
    
    def __addEdgesBasedOnProximity(self,normaliseLimit):
        allNodes = list(self.model.nodes)
        newNodes = []
        print("\tnumber of nodes =",len(allNodes))
        print("\t", end='')
        for i, n1 in enumerate(allNodes):
            if(i%10==0): print(i, end=' ')
            for n2 in allNodes[i+1:]:
                if not self.model.has_edge(n1, n2):
                    commonN = len(list(nx.common_neighbors(self.model, n1, n2)))
                    if commonN > 0:
                        newNodes.append((n1, n2, self.__normalise(commonN, 0, normaliseLimit)))
        self.model.add_weighted_edges_from(newNodes, weight ='prob')
        print('')
    
    def __classifyInstance1(self, source, sink):
        if self.model.has_edge(source, sink):
            return self.model[source][sink]['prob']
        else:
            return 0
    
    #####################################################################
    # Attempt 2: Kaggle Score = 0.54019 (MSE =  0.16916446409631972)
    #####################################################################
    
    def __trainAttempt2(self, graph):
        self.__buildProbabilityGraph(graph)
        print('iteration 0: number of edges = ', len(self.model.edges))
        self.__addEdgesBasedOnProximity(12)
        print('iteration 1: number of edges = ', len(self.model.edges))    
    
    def __predictAttempt2(self, edgeList):
        return edgeList.apply(
            lambda i: self.__classifyInstance2(str(i.Source), str(i.Sink)),
            axis=1)
    
    def __classifyInstance2(self, source, sink):
        if self.model.has_edge(source, sink):
            return self.model[source][sink]['prob']
        else:
            try:
                dist = len(list(nx.dijkstra_path(trainGraph, source=source, target=sink))) - 1
                return 1/dist
            except:
                return 0

    #####################################################################
    # Attempt 3: Kaggle Score = 0.57135 (MSE =  0.1602347332838925)
    #####################################################################
    
    def __trainAttempt3(self, graph):
        self.__buildProbabilityGraph(graph)
        print('iteration 0: number of edges = ', len(self.model.edges))
    
    def __predictAttempt3(self, edgeList):
        return edgeList.apply(
            lambda i: self.__classifyInstance3(str(i.Source), str(i.Sink)),
            axis=1)
    
    def __classifyInstance3(self, source, sink):
        if self.model.has_edge(source, sink):
            return self.model[source][sink]['prob']
        else:
            try:
                dist = len(list(nx.dijkstra_path(trainGraph, source=source, target=sink))) - 1
                return 1/dist
            except:
                return 0
    
            
    #####################################################################
    # Helper Functions
    #####################################################################

    def __buildProbabilityGraph(self, graph):
        self.model = nx.Graph()
        for (source, sink) in graph.edges:
            freq = graph[source][sink]['frequency']
            prob = self.__normalise(freq, 0, 4) #TODO: remove hardcoded
            self.model.add_edge(source, sink, prob = prob)
    
    def __normalise(self, v, minF, maxF):
        result = (v - minF) / (maxF - minF)
        if result > 1: return 1
        elif result <= 0: return 0
        else: return 1

In [5]:
classifier = EdgeAddClassifier()
classifier.train(trainGraph)

iteration 0: number of edges =  16036
	number of nodes = 3767
	0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 470 480 490 500 510 520 530 540 550 560 570 580 590 600 610 620 630 640 650 660 670 680 690 700 710 720 730 740 750 760 770 780 790 800 810 820 830 840 850 860 870 880 890 900 910 920 930 940 950 960 970 980 990 1000 1010 1020 1030 1040 1050 1060 1070 1080 1090 1100 1110 1120 1130 1140 1150 1160 1170 1180 1190 1200 1210 1220 1230 1240 1250 1260 1270 1280 1290 1300 1310 1320 1330 1340 1350 1360 1370 1380 1390 1400 1410 1420 1430 1440 1450 1460 1470 1480 1490 1500 1510 1520 1530 1540 1550 1560 1570 1580 1590 1600 1610 1620 1630 1640 1650 1660 1670 1680 1690 1700 1710 1720 1730 1740 1750 1760 1770 1780 1790 1800 1810 1820 1830 1840 1850 1860 1870 1880 1890 1900 1910 1920 1930 1940 1950 1960 1970 1980 1990 2000 2010 2020 2030 2040 2050 2060 2070 2080 209

## 4. Predict

In [6]:
test = pd.read_csv('test-public.csv')
test['Predicted'] = classifier.predict(test)
test[['Id', 'Predicted']].to_csv('results/EdgeAddClassifier.csv', index=False)

## 5. Evaluate

In [7]:
sample = pd.read_csv('sample.csv')
yTrue = sample[['Predicted']]
yPred = test[['Predicted']]
print('MSE = ', mean_squared_error(yTrue, yPred))

MSE =  0.3244069239500406


In [13]:
sum(list(yPred.values))

array([734], dtype=int64)